In [ ]:
!pip install - r requirements.txt


In [ ]:
import azure.core
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException


In [ ]:
resource_name = "sentiment-rg"
workspace_name = "sentiment-ws"
subscriptionID = "<your-subscription-id>"
aml_compute_target = "sentiment-compute"
experiment_name = 'sentiment-experiment'


# Create or get workspace


In [ ]:
try:
    ws = Workspace.from_config()
    print('Workspace is already exist')
except:
    try:
        ws = Workspace.get(workspace_name,
                           resource_group=resource_name,
                           subscription_id=subscriptionID,
                           location="East US")
        ws.write_config('.azureml')
        print('Workspace is found')
    except:
        ws = Workspace.create(workspace_name,
                              resource_group=resource_name,
                              create_resource_group=True,
                              subscription_id=subscriptionID,
                              location="East US")
        ws.write_config('.azureml')
        print('Workspace is created')


# Create a compute cluster


In [ ]:
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :", aml_compute_target)

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=1,
                                                                max_nodes=4,
                                                                idle_seconds_before_scaledown=3000)
    aml_compute = ComputeTarget.create(
        ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

print("Azure Machine Learning Compute attached now")


# Create an Experiment


In [ ]:
exp = Experiment(ws, experiment_name)


# Create and execute job


In [ ]:
env = Environment.from_pip_requirements(
    name="sentiment-env", file_path="./requirements.txt")
config = ScriptRunConfig(source_directory="./model", script="train.py",
                         compute_target=aml_compute_target, environment=env)
execution = exp.submit(config)
execution.wait_for_completion(show_output=True)


# Getting the registered the model


Model is registered at the end of the train file


In [ ]:
# get the model registered by its name
from azureml.core import Model
model = Model(ws, "sentiment_model")
model


# Deploy the model


In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

myenv = Environment.from_pip_requirements(
    name="sentiment-env", file_path="./requirements.txt")
myenv.register(workspace=ws)
inference_config = InferenceConfig(
    entry_script='classify.py', source_directory='./endpoint/', environment=myenv)


In [ ]:
service = Model.deploy(ws, "sentiment-model",
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=AciWebservice.deploy_configuration(
                           cpu_cores=1, memory_gb=1),
                       overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)


# Testing the Deployed Model


In [ ]:
from azureml.core import Webservice
service = Webservice(workspace=ws, name="sentiment-model")
print(service.scoring_uri)
print(service.swagger_uri)


In [ ]:
import pandas as pd

df = pd.read_csv("./model/sample.csv")
df = df.sample(5)
df.iloc[:, 0].tolist()


In [ ]:
import requests
import json
headers = {'content-Type': 'application/json'}

test = {"data": df.iloc[:, 5].tolist()}
test = json.dumps(test)
response = requests.post(service.scoring_uri, test, headers=headers)
response.text
